In [1]:
import pandas as pd   #导入三件套
import numpy as np
import matplotlib.pyplot as plt

# Pandas数据清洗
学习自：https://edu.cda.cn/my/course/1441
源码与数据：.\data\data_cleaning

数据清洗：对数据进行重新审查和校验的过程，目的在于删除重复信息、纠正存在的错误，并提供数据一致性。

## 数据分析的流程
数据的读写->数据的探索与描述->数据简单预处理->针对实际情况进行数据处理

# 案例一：文本字符串的清洗

### 1，数据的读写

In [2]:
df=pd.read_csv("data/data_cleaning/qunar_freetrip.csv"   #导入数据
               ,index_col=0)   #第0列作为索引
df.head(2)

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
0,哈尔滨,北海,2208.0,650.0,哈尔滨-北海3天2晚 | 入住北海祥丰嘉年华大酒店 + 春秋航空往返机票,北海祥丰嘉年华大酒店 舒适型 4.7分/5分,标准双人间(双床) 双床 不含早 1间2晚,春秋航空 9C8741,直飞,17:10-21:50,春秋航空 9C8742,直飞,10:20-15:05
1,成都,泸沽湖,1145.0,376.0,成都-泸沽湖3天2晚 | 入住7天酒店丽江古城中心店 + 成都航空往返机票,7天酒店丽江古城中心店 经济型 4.0分/5分,经济房-不含早-限时特... 其他 不含早 1间2晚,成都航空 EU2237,直飞,19:45-21:20,成都航空 EU2738,直飞,23:30-01:05


### 2,数据探索与描述

In [3]:
#查看数据形状
df.shape

(5100, 13)

In [4]:
#查看数据结构
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5100 entries, 0 to 5099
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   出发地     5098 non-null   object 
 1    目的地    5099 non-null   object 
 2   价格      5072 non-null   float64
 3   节省      5083 non-null   float64
 4   路线名     5100 non-null   object 
 5   酒店      5100 non-null   object 
 6   房间      5100 non-null   object 
 7   去程航司    5100 non-null   object 
 8   去程方式    5100 non-null   object 
 9   去程时间    5100 non-null   object 
 10  回程航司    5100 non-null   object 
 11  回程方式    5100 non-null   object 
 12  回程时间    5100 non-null   object 
dtypes: float64(2), object(11)
memory usage: 557.8+ KB


发现索引有5100行，但有些数据并没有那么多，这就代表一些数据存在空值

In [5]:
#快速查看数据的描述统计
df.describe()   #只会显示数字列统计结果

,价格,节省
count,5072.000000,5083.000000
mean,1765.714905,474.139878
std,2580.129644,168.893780
min,578.000000,306.000000
25%,1253.000000,358.000000
50%,1632.000000,436.000000
75%,2028.250000,530.000000
max,179500.000000,3500.000000


价格与节省的数据量（count）小于5100，代表这两个是有缺失值存在；
价格与节省的最大值与%75值大太多，代表有概率存在异常值

### 3,数据的简单预处理

In [6]:
df.columns  #查看列名

Index(['出发地 ', ' 目的地', '价格 ', '节省', '路线名', '酒店', '房间', '去程航司', '去程方式', '去程时间',
       '回程航司', '回程方式 ', '回程时间'],
      dtype='object')

⚠注意，有些列名存在空格，可能会导致异常

In [7]:
col = df.columns.values   #获取列名
col

array(['出发地 ', ' 目的地', '价格 ', '节省', '路线名', '酒店', '房间', '去程航司', '去程方式',
       '去程时间', '回程航司', '回程方式 ', '回程时间'], dtype=object)

In [8]:
print(col[0])
print(col[0].strip())   #strip方法可以去除字符串头尾字符（默认空格）但只能一次

出发地 
出发地


In [9]:
[x for x in col]  #列表推导式原理（遍历-》处理-》返回一个列表）

['出发地 ',
 ' 目的地',
 '价格 ',
 '节省',
 '路线名',
 '酒店',
 '房间',
 '去程航司',
 '去程方式',
 '去程时间',
 '回程航司',
 '回程方式 ',
 '回程时间']

In [10]:
df.columns = [x.strip() for x in col]   #批量取空格，写循环也行,并赋值给列索引

In [11]:
df.columns  #发现空格已被去除

Index(['出发地', '目的地', '价格', '节省', '路线名', '酒店', '房间', '去程航司', '去程方式', '去程时间',
       '回程航司', '回程方式', '回程时间'],
      dtype='object')

### 4，重复值的处理
#### 检查重复值duplicated()
Duplicated函数功能：查找并显示数据表中的重复值   
这里需要注意的是：    
- 当两条记录中所有的数据都相等时duplicated函数才会判断为重复值(一整行每列数据都完全相等)       
- duplicated支持从前向后(默认)(first)(后面的判断为重复值);和从后向前(last)(前面的判断为重复值)两种重复值查找模式   
- 默认是从前向后进行重复值的查找和判断，也就是后面的条目在重复值判断中显示为True  

In [12]:
df.duplicated()   #返回每个元素是否是重复值（布尔类型）

0       False
1       False
2       False
3       False
4       False
        ...  
5095     True
5096    False
5097    False
5098    False
5099    False
Length: 5100, dtype: bool

In [13]:
df[df.duplicated()].head(2)   #重复值

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
454,广州,黄山,1871.0,492.0,广州-黄山3天2晚 | 入住黄山汤口醉享主题酒店 + 南方航空往返机票,黄山汤口醉享主题酒店 舒适型 4.8分/5分,睫毛弯弯(大床) 大床 不含早 1间2晚,南方航空 CZ3627,直飞,19:20-21:15,南方航空 CZ3628,直飞,22:05-23:50
649,济南,长沙,1134.0,360.0,济南-长沙3天2晚 | 入住长沙喜迎宾华天大酒店 + 山东航空往返机票,长沙喜迎宾华天大酒店 高档型 3.7分/5分,特惠双间(特惠抢购)(... 双床 不含早 1间2晚,山东航空 SC1185,直飞,18:40-20:50,山东航空 SC1186,直飞,10:20-12:15


In [14]:
df.duplicated().sum()   #有几个重复值

100

#### 删除重复值drop_duplicates()   
- drop_duplicates函数功能是：删除数据表中的重复值，判断标准和逻辑与duplicated函数一样 
- drop_duplicates函数有一个inplace参数，默认为False(不修改原数据),True(修改原数据)

In [15]:
df.drop_duplicates().head(1)  #注意，原始数据重复值没有删除,输出的是删除后数据

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
0,哈尔滨,北海,2208.0,650.0,哈尔滨-北海3天2晚 | 入住北海祥丰嘉年华大酒店 + 春秋航空往返机票,北海祥丰嘉年华大酒店 舒适型 4.7分/5分,标准双人间(双床) 双床 不含早 1间2晚,春秋航空 9C8741,直飞,17:10-21:50,春秋航空 9C8742,直飞,10:20-15:05


In [16]:
df.duplicated().sum()   #重复值计数

100

In [17]:
df.drop_duplicates(inplace=True)   #删除重复值

In [18]:
df.duplicated().sum()    #重复值计数

0

In [19]:
df[4997:] #输出最后几行

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
5097,天津,丽江,1616.0,426.0,天津-丽江3天2晚 | 入住丽江凡间度假连锁客栈青旅店 + 天津航空/首都航空往...,丽江凡间度假连锁客栈青旅店 经济型 4.5分/5分,大床房-预付 大床 不含早 1间2晚,天津航空 GS7861,直飞,16:25-19:45,首都航空 JD5739,直飞,07:50-10:50
5098,大连,重庆,1703.0,446.0,大连-重庆3天2晚 | 入住重庆酉阳锦宏大酒店 + 华夏航空/山东航空往返机票,重庆酉阳锦宏大酒店 舒适型 4.0分/5分,特惠房(大床) 大床 不含早 1间2晚,华夏航空 G52762,经停,18:25-23:30,山东航空 SC4837,经停,07:00-11:30
5099,天津,哈尔滨,1192.0,356.0,天津-哈尔滨3天2晚 | 入住哈尔滨钰轩酒店中央大街店 + 奥凯航空/福州往返机票,哈尔滨钰轩酒店中央大街店 舒适型 4.0分/5分,标准大床房 大床 大床 双早 1间2晚,奥凯航空 BK2918,直飞,21:10-23:25,福州 FU6556,直飞,11:35-13:45


⚠注意：观察最后几行索引，可见只是去除了重复值，索引还是保留之前的

In [20]:
df.index=(range(df.shape[0]))  #重置索引(返回一个0-列表行数（不包含）的序列进行赋值)

In [21]:
df[4998:] #输出最后几行

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
4998,大连,重庆,1703.0,446.0,大连-重庆3天2晚 | 入住重庆酉阳锦宏大酒店 + 华夏航空/山东航空往返机票,重庆酉阳锦宏大酒店 舒适型 4.0分/5分,特惠房(大床) 大床 不含早 1间2晚,华夏航空 G52762,经停,18:25-23:30,山东航空 SC4837,经停,07:00-11:30
4999,天津,哈尔滨,1192.0,356.0,天津-哈尔滨3天2晚 | 入住哈尔滨钰轩酒店中央大街店 + 奥凯航空/福州往返机票,哈尔滨钰轩酒店中央大街店 舒适型 4.0分/5分,标准大床房 大床 大床 双早 1间2晚,奥凯航空 BK2918,直飞,21:10-23:25,福州 FU6556,直飞,11:35-13:45


## 异常值的处理

In [22]:
df.describe().T   #查看统计结果（.T可以调换行/列）

,count,mean,std,min,25%,50%,75%,max
价格,4972.0,1767.782381,2604.329780,578.0,1253.0,1633.0,2031.0,179500.0
节省,4983.0,474.490869,169.148391,306.0,358.0,436.0,532.0,3500.0


In [23]:
#找出价格异常值（用数据是否大于三倍的标准方差判断）
sta = (df["价格"]-df["价格"].mean())/df["价格"].std()  #（值-平均值）/标准差
df[sta.abs()>3]   #返回绝对值大于三倍标准差的值

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
2763,杭州,九寨沟,179500.0,538.0,杭州-九寨沟3天2晚 | 入住九寨沟九乡宾馆 + 成都航空/长龙航空往返机票,九寨沟九乡宾馆 舒适型 4.3分/5分,特惠房(双床) 双床 不含早 1间2晚,成都航空 EU2206,经停,20:30-01:00,长龙航空 GJ8680,经停,20:25-00:50


In [24]:
#找出节省异常值（用节省金额大于价格作为判断）
(df["节省"] > df["价格"]).sum()   #可见数据中存在三个节省大于价格的数据，这通常不符合实际

3

In [25]:
df[df["节省"]>df["价格"]]

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
2904,武汉,西安,949.0,3500.0,武汉-西安3天2晚 | 入住西安西稍门大酒店 + 东方航空往返机票,西安西稍门大酒店 舒适型 3.3分/5分,标准间B(丝路之旅)(... 双床 不含早 1间2晚,东方航空 MU2194,直飞,21:50-23:30,东方航空 MU2462,直飞,19:35-21:20
3108,济南,大连,911.0,3180.0,济南-大连3天2晚 | 入住普兰店科洋大酒店 + 山东航空/厦门航空往返机票,普兰店科洋大酒店 舒适型 4.4分/5分,大床房(限量促销) 大床 不含早 1间2晚,山东航空 SC4916,直飞,19:45-20:50,厦门航空 MF8042,直飞,13:10-14:20
3660,沈阳,青岛,924.0,3200.0,沈阳-青岛3天2晚 | 入住星程酒店青岛台东步行街店 + 青岛航/南方航空往返机票,星程酒店青岛台东步行街店 舒适型 4.2分/5分,大床房(内宾)(提前1... 大床 不含早 1间2晚,青岛航 QW9780,直飞,22:35-00:10,南方航空 CZ6568,直飞,20:55-22:35


- 对于建模来说，可以直接删除异常值，且不影响
- 对于业务来说，直接删除可能会导致一些数据的丢失
- 具体需要按实际情况做判断

In [26]:
#将异常值数据结合起来，记录索引
deindex = pd.concat([df[df["节省"]>df["价格"]],df[sta.abs()>3]]).index   
deindex

Int64Index([2904, 3108, 3660, 2763], dtype='int64')

In [27]:
#根据索引删除
df.drop(deindex,inplace=True)  #drop方法也有inplace属性，作用和前者一样

## 6.缺失值的处理
- df.isnull()  #查看缺失值
- df.notnull() #查看不是缺失值的数据
- df.dropna()  #删除缺失值
- df.fillna()  #填补缺失值 

In [28]:
df.isnull().head(6)  #返回每一行所有数据是否是缺失值

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False


In [29]:
df.isnull().sum()    #可以是由sum进行统计，返回的是每一个列缺失值的数量

出发地      2
目的地      1
价格      28
节省      17
路线名      0
酒店       0
房间       0
去程航司     0
去程方式     0
去程时间     0
回程航司     0
回程方式     0
回程时间     0
dtype: int64

#### 处理出发地的空值数据
利用路线名中的数据进行填充

In [30]:
df[df.出发地.isnull()]   #所有出发低地为空的数据   （注意，每个列可以当作df的一个属性对待）

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
1850,NaN,烟台,647.0,348.0,大连-烟台3天2晚 | 入住烟台海阳黄金海岸大酒店 + 幸福航空/天津航空往返机票,烟台海阳黄金海岸大酒店 3.7分/5分,海景标准间(内宾)[双... 双床 双早 1间2晚,幸福航空 JR1582,直飞,10:05-11:05,天津航空 GS6402,直飞,16:30-17:25
1915,NaN,西安,1030.0,326.0,济南-西安3天2晚 | 入住西安丝路秦国际青年旅舍钟楼回民街店 + 华夏航空往返...,西安丝路秦国际青年旅舍钟楼回民街店 经济型 4.4分/5分,标准间（独卫）-吃货天... 双床 不含早 1间2晚,华夏航空 G54963,直飞,07:10-08:55,华夏航空 G58858,直飞,23:10-00:55


⚠发现虽然出发地丢失了，但路线名却还在，出发地与目的地都可在里面找到

In [31]:
#空值原有的数据可以通过路线名第一个"-"前面的字符获取到（视频老师的那个方法有局限性）
[str(x).split("-")[0] for x in df.loc[df.出发地.isnull(),'路线名']]

['大连', '济南']

In [32]:
#替换
df.loc[df.出发地.isnull(),'出发地'] = [str(x).split("-")[0] for x in df.loc[df.出发地.isnull(),'路线名']]

In [33]:
df[df.出发地.isnull()]   #所有出发地为空的数据

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间


#### 处理目的地的空值数据
利用路线名中的数据进行填充

In [34]:
df[df.目的地.isnull()]  #所有目的地为空的数据

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间
1860,深圳,NaN,2149.0,494.0,深圳-大连3天2晚 | 入住大连黄金山大酒店 + 南方航空/东海往返机票,大连黄金山大酒店 舒适型 3.4分/5分,标准间 大/双床 不含早 1间2晚,南方航空 CZ6833,直飞,09:10-12:40,东海 DZ6242,经停,12:40-18:00


In [35]:
import jieba    #路线名目的地后没有-，所以不能用之前的方法，这里我使用jieba库
jieba.lcut(str(df.loc[df.目的地.isnull(),'路线名'].values))[4]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.088 seconds.
Prefix dict has been built successfully.


'大连'

In [36]:
#替换
df.loc[df.目的地.isnull(),'目的地'] = jieba.lcut(str(df.loc[df.目的地.isnull(),'路线名'].values))[4]

In [37]:
df[df.目的地.isnull()]  #所有目的地为空的数据

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间


#### 处理价格/节省的空值数据
利用平均值进行填充

In [38]:
#返回平均值(不保留小数)
round(df['价格'].mean(),0)   #round函数可以保留n为小数（！返回数据类型是浮点型）

1733.0

In [39]:
#处理价格缺失值
df['价格'].fillna(round(df['价格'].mean(),0),inplace=True)

In [40]:
#处理节省缺失值
df['节省'].fillna(round(df['节省'].mean(),0),inplace=True)

In [41]:
df.isnull().sum()

出发地     0
目的地     0
价格      0
节省      0
路线名     0
酒店      0
房间      0
去程航司    0
去程方式    0
去程时间    0
回程航司    0
回程方式    0
回程时间    0
dtype: int64

至此空值已经全部被处理

## 处理文本型数据

In [42]:
df.酒店.head(10)

0       北海祥丰嘉年华大酒店 舒适型 4.7分/5分
1      7天酒店丽江古城中心店 经济型 4.0分/5分
2           沈阳中煤宾馆 舒适型 4.5分/5分
3          红原芸谊大酒店 舒适型 4.6分/5分
4        天津逸海明珠大酒店 高档型 4.1分/5分
5        青岛康大豪生大酒店 豪华型 4.6分/5分
6         阿坝松潘松林酒店 舒适型 4.3分/5分
7    重庆运通假日酒店汽博中心店 舒适型 4.3分/5分
8     维也纳酒店南京南站汇景店 舒适型 4.4分/5分
9    厦门泊旅时尚酒店会展中心店 舒适型 4.0分/5分
Name: 酒店, dtype: object

In [43]:
#提取酒店的评分
#pandas的str.extract https://blog.csdn.net/winnie_shi/article/details/88240907
df.酒店.str.extract('(\d\.\d)分/5分',expand=True)[:10]  
#expand=True返回一个DataFrame，expand=False返回一个Series

,0
0,4.7
1,4.0
2,4.5
3,4.6
4,4.1
5,4.6
6,4.3
7,4.3
8,4.4
9,4.0


In [44]:
#添加到数据中
df['酒店评分'] = df.酒店.str.extract('(\d\.\d)分/5分',expand=False)
#括号里面的是你要返回的内容，外面的是用来定为位置
df.head(1)

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间,酒店评分
0,哈尔滨,北海,2208.0,650.0,哈尔滨-北海3天2晚 | 入住北海祥丰嘉年华大酒店 + 春秋航空往返机票,北海祥丰嘉年华大酒店 舒适型 4.7分/5分,标准双人间(双床) 双床 不含早 1间2晚,春秋航空 9C8741,直飞,17:10-21:50,春秋航空 9C8742,直飞,10:20-15:05,4.7


In [45]:
#提起酒店的等级
df.酒店.str.extract(' (.+) ',expand=False)[:5]

0    舒适型
1    经济型
2    舒适型
3    舒适型
4    高档型
Name: 酒店, dtype: object

In [46]:
#添加到数据中
df['酒店等级'] = df.酒店.str.extract(' (.+) ',expand=False)
#+号表示的是贪婪模式，也就是所有的数据都要提取出来
df.head(1)

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间,酒店评分,酒店等级
0,哈尔滨,北海,2208.0,650.0,哈尔滨-北海3天2晚 | 入住北海祥丰嘉年华大酒店 + 春秋航空往返机票,北海祥丰嘉年华大酒店 舒适型 4.7分/5分,标准双人间(双床) 双床 不含早 1间2晚,春秋航空 9C8741,直飞,17:10-21:50,春秋航空 9C8742,直飞,10:20-15:05,4.7,舒适型


In [47]:
#提取并添加天数信息
df['天数']=df.路线名.str.extract('(\d+)天\d晚',expand=False)
df.head(1)

,出发地,目的地,价格,节省,路线名,酒店,房间,去程航司,去程方式,去程时间,回程航司,回程方式,回程时间,酒店评分,酒店等级,天数
0,哈尔滨,北海,2208.0,650.0,哈尔滨-北海3天2晚 | 入住北海祥丰嘉年华大酒店 + 春秋航空往返机票,北海祥丰嘉年华大酒店 舒适型 4.7分/5分,标准双人间(双床) 双床 不含早 1间2晚,春秋航空 9C8741,直飞,17:10-21:50,春秋航空 9C8742,直飞,10:20-15:05,4.7,舒适型,3


# 其他相关方法